# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## Imports

In [1]:
from IPython.display import Markdown

## Today

In [2]:
import datetime
today = str(datetime.datetime.now())[0:10]
today

'2024-07-30'

## 000_bot

### API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api

In [3]:
from omegaconf import OmegaConf

tokens = OmegaConf.create("""
METACULUS_TOKEN: xx
OPENAI_API_KEY: yy
OPENAI_MODEL: gpt-4o
PERPLEXITY_API_KEY: zz
PERPLEXITY_MODEL: llama-3-sonar-large-32k-online""")

token_fn = "tokens.yaml"
# OmegaConf.save(config=tokens, f=token_fn)
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

### LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [4]:
import datetime
import json
import os
import requests
import re
from openai import OpenAI
from tqdm import tqdm

In [5]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

### IFP

In [6]:
class IFP:

    def __init__(self, question_id):
        self.question_id = question_id
        self.question_details = get_question_details(self.question_id)
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")   
        self.title = self.question_details["title"]
        self.resolution_criteria = self.question_details["resolution_criteria"]
        self.background = self.question_details["description"]
        self.fine_print = self.question_details["fine_print"]

    def report(self):
        rpt = f"""
The future event is described by this question: [ {self.title} ]
The resolution criteria are: [ {self.resolution_criteria} ]
The background is: [ {self.background} ]"""
        if self.fine_print:
            rpt += f"""
The fine print is: [ {self.fine_print} ]"""
        return rpt

### LLMs

In [7]:
class LLM:
    def __init__(self, system_role):
        self.messages = [{"role": "system", "content": system_role}]

    def chat(self, query):
        self.messages.append({"role": "user", "content": query})
        text = self.message()
        self.messages.append({"role": "assistant", "content": text})
        return text

class Perplexity(LLM):
    def message(self):
        url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
            "content-type": "application/json"  }
        payload = {"model": config.PERPLEXITY_MODEL, "messages": self.messages }
        response = requests.post(url=url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

class ChatGPT(LLM):
    def __init__(self, system_role):
        super().__init__(system_role)
        self.client = OpenAI(api_key=config.OPENAI_API_KEY)

    def message(self):
        chat_completion = self.client.chat.completions.create(
            model=config.OPENAI_MODEL,
            messages= self.messages)
        return chat_completion.choices[0].message.content

### Test questions

### Agents

In [8]:
class Agent:
    def __init__(self, system_role, llm):
        self.llm = llm(system_role)

    def chat(self, prompt):
        return self.llm.chat(prompt)

## Rate Analyzer(TBD: ChatGPT is better)

#### Researcher

In [9]:
class Researcher(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are an open source intelligence analyst.
You summarize news related to questions about events.
A question about an event is formatted as |id|question|criteria|background|fineprint|.
You will find on the web and report any reliable information you can gather about the question.
Do not make an assessment of probability.
Do not repeat information provided to you already in the prompt."""
        super().__init__(self.system_role, llm)

    def recherche(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.news = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)

    def research(self, ifps):
        for ifp in tqdm(ifps):
            self.recherche(ifps[ifp])

### Question relator

In [10]:
class QuestionRelator(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are prompted with list of forecasting questions, each with an id and a title.
Label each question with an underlying event.
If the questions are for the same event, use the same name for the underlying event.
Please return as separate lines formatted as |event|id|title|, do not add any other formatting.
"""
        super().__init__(self.system_role, llm)

    def relate(self, ifps):
        prompt = '\n'.join([f"{ifp.question_id}: {ifp.title}" for ifp in ifps.values()])
        KL = self.chat(prompt)
        K1 = [x.split('|') for x in KL.split('\n')]
        K2 = [(int(id),event) for _,event,id,_,_ in K1]
        for id,event in K2:
            ifps[id].event = event
            print(id, event)

#### Superforecaster

In [11]:
class Superforecaster(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are a superforecaster.  
You assign a probability to questions about events.
Questions are given as separate groups of lines formatted as |FORECAST|event|id|question|news|criteria|background|fineprint|.
Groups are separated by '^^^'.
Questions which are about the same event should be assigned consistent probabilities.
Reply to questions with |ASSESSMENT|id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and rationale is your reasoning for the forecast.
Separate each question with '^^^'.
Do not add any additional headings or group labels or other formatting.
After your initial forecast you may receive feedback of form |CRITIC|id|feedback|.
Reply to each feedback with |id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and 
and rationale is a revised assessment which may be adjusted from a prior assessment due the feedback unless the feedback is "I concur".
"""
        super().__init__(self.system_role, llm)

    def forecast(self, ifps):
        prompt = '^^^'.join([f"FORECAST|{ifp.event}|{ifp.question_id}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|" for ifp in ifps.values()])
        self.F0 = self.chat(prompt)
        self.F1 = [x.strip().replace('\n', '') for x in self.F0.split('^^^')]
        self.F2 = [x.split('|') for x in self.F1] 
        self.F3 = [[x for x in y if x] for y in self.F2]
        self.F4 = [(int(id),int(forecast),rationale) for _, id, forecast, rationale in self.F3]
        for id, forecast, rationale in self.F4:
            ifps[id].forecast = forecast
            ifps[id].rationale = rationale
            print(id, forecast, rationale)

    def reassess(self, ifp):
        prompt = f"|CRITIC|{ifp.question_id}|{ifp.feedback}|"
        self.R0 = self.chat(prompt)
        id,fcst,rationale = [x for x in self.R0.strip().split('|') if x]
        id = int(id)
        fcst = int(fcst)
        ifps[id].forecast = fcst
        ifps[id].rationale = rationale
        print(id, fcst, rationale)

#### Critic

In [12]:
class Critic (Agent):
    def __init__(self, llm):

        self.system_role = f"""
You a very smart and worldly person reviewing a superforecaster's assignment of probabilities to events.
You will receive an event with probabilities given as |event|id|question|zz|rationale|news|criteria|background|fineprint|.
zz is an integer probability from 1 to 99 and rationale is the student's logic for assigning probability of zz.
You will reply with a line |id|feedback| where feedback is "I concur" if you see no problem with the rationale and zz otherwise presents possible problems with the rationale and zz.
"""
        super().__init__(self.system_role, llm)

    def feedback(self, ifp):
        prompt = f"|{ifp.event}|{ifp.question_id}|{ifp.title}|{ifp.forecast}|{ifp.rationale}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.fb = self.chat(prompt)
        self.fb1 = self.fb.split('|')
        self.fb2 = [x for x in self.fb1 if x]
        try:
            id,feedback = self.fb2
            ifps[int(id)].feedback = feedback
            print(id, feedback)
        except:
            print('problem', self.fb2)
            ifps[int(id)].feedback = 'I concur'

## Summarizer

Add an agent to summarize the back and forth between critic and forecaster into a single cogent rationale that incorporates all that was discussed.

### Forecasting process

In [13]:
def forecasting(ifps):
    max_tries = 4
    
    analyst = Researcher(ChatGPT)
    analyst.research(ifps)
    
    qr = QuestionRelator(ChatGPT)
    qr.relate(ifps)
    
    sf = Superforecaster(ChatGPT)
    sf.forecast(ifps)
    
    for ifp in ifps.values():
        print("Refining", ifp.question_id)
        ifp.feedback = ''
        critic = Critic(Perplexity)
        for i in range(max_tries):
            print("Pass", i, "of", max_tries, "on", ifp.question_id)
            if 'I concur' in ifp.feedback:
                break
            critic.feedback(ifp)
            if 'I concur' in ifp.feedback:
                break
            sf.reassess(ifp)
        print("===============================================")

    for ifp in ifps.values():
        print(ifp.question_id, ifp.title)
        print("Forecast", ifp.forecast)
        print("Rationale", ifp.rationale, '\n')

In [14]:
def upload(ifp):
    post_question_prediction(ifp.question_id, ifp.forecast)
    post_question_comment(ifp.question_id, ifp.rationale)

In [15]:
def uploads(ifps):
    for ifp in ifps.values():
        upload(ifp)

## Daily forecast

### Get IFP ids

In [16]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
# today_ids = [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872] # 08JUL24
# today_ids = [26006, 25936, 25935, 25934, 25933, 26004, 26005] # 09JUL24
# today_ids = [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958] # 10JUL24
# today_ids = [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024] # 11JUL24
# today_ids = [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102] # 12JUL24
# today_ids = [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159] # 15JUL24
# today_ids = [26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196] # 16JUL24
# today_ids = [26210, 26211, 26212, 26213, 26214, 26215, 26216] # 17JUL24
# today_ids = [26232, 26233, 26234, 26235, 26236] # 18JUL24
# today_ids = [26302, 26303, 26304, 26305, 26306, 26307] # 19JUL24
# today_ids = [26387, 26388, 26389, 26390, 26391, 26392] # 22JUL24
# today_ids = [26404, 26405, 26406, 26407, 26408] # 23JUL24
# today_ids = [26550, 26551, 26552, 26553, 26554, 26555] # 24JUL24
# today_ids = [26568, 26569, 26570, 26571, 26572, 26573, 26574, 26575, 26576, 26577] # 25JUL24
# today_ids = [26638, 26639, 26640, 26641, 26642, 26643, 26644, 26645, 26646] # 26JUL24
# today_ids = [26665, 26666, 26667, 26668, 26669, 26670, 26671, 26683] # 29JUL24
# today_ids = [26700, 26701, 26702, 26703, 26704, 26705, 26706] # 30JUL24

In [18]:
today_ids

[26700, 26701, 26702, 26703, 26704, 26705, 26706]

## Forecast

In [19]:
ifps = {id: IFP(id) for id in today_ids}

In [20]:
forecasting(ifps)

 14%|██████▍                                      | 1/7 [00:02<00:14,  2.37s/it]

26700 Will the total market cap of all cryptocurrencies be more than $1.5 trillion on September 29, 2024?
As of now, there isn't a way to provide information about the total market capitalization of all cryptocurrencies on September 29, 2024. You can monitor the total market cap of cryptocurrencies via CoinMarketCap's charts [here](https://coinmarketcap.com/charts/) as the date approaches. You might also consider checking news and updates regularly around that time frame for any forecasts or changes in the cryptocurrency market.


 29%|████████████▊                                | 2/7 [00:05<00:13,  2.66s/it]

26701 Will the total market cap of all cryptocurrencies be more than $2 trillion on September 29, 2024?
As of now, it's not possible to provide the total market capitalization of all cryptocurrencies on September 29, 2024. However, you can monitor the total market cap of cryptocurrencies via CoinMarketCap's charts [here](https://coinmarketcap.com/charts/). To get the most accurate and up-to-date information, you should check this resource as that date approaches.

Additionally, you may consider keeping an eye on relevant news sources, economic reports, regulatory changes, and technological advancements in the cryptocurrency world, as these factors can significantly influence the market cap.


 43%|███████████████████▎                         | 3/7 [00:07<00:09,  2.35s/it]

...

 86%|██████████████████████████████████████▌      | 6/7 [00:18<00:03,  3.78s/it]

26705 Will Apple announce an iPhone with a graphene thermal system before October 1, 2024?

As of now, Apple has not officially announced an iPhone with a graphene thermal system. However, according to the provided [MacRumors](https://www.macrumors.com/roundup/iphone-16/) source, there is speculation and expectation that the upcoming iPhone 16 models might include such a system.

To determine if this occurs before October 1, 2024, you should monitor Apple's official announcements and press releases closely. Apple's official announcements are typically made during their worldwide developer conferences (WWDC) and special event keynotes, which are streamed and widely covered by major tech news outlets.

Here are some actions you can take to stay updated:

1. Follow Apple's [official newsroom](https://www.apple.com/newsroom/) for the latest press releases and product announcements.
2. Keep an eye on major tech news websites such as MacRumors, The Verge, and TechCrunch, which frequently rep

100%|█████████████████████████████████████████████| 7/7 [00:21<00:00,  3.10s/it]

26706 Before October 1, 2024, will Stripe announce on the news section of its website that it is planning an IPO?
As of now, there have been no announcements on the [news section of Stripe’s website](https://stripe.com/newsroom/news) regarding plans for an IPO. To determine if this occurs before October 1, 2024, you should regularly check the specified news section of Stripe's website for any updates.

To stay updated and potentially catch the announcement if it happens, you can:

- Regularly visit the [news section of Stripe’s website](https://stripe.com/newsroom/news).
- Set up Google Alerts for news on Stripe’s IPO plans.
- Follow financial news and major business news websites for any hints or unofficial information that might indicate an upcoming announcement.

If Stripe announces an IPO in the [news section](https://stripe.com/newsroom/news) of its website before October 1, 2024, this question will resolve as Yes. If there is no such announcement by that date, it will resolve as 

26700 Cryptocurrency Market Cap
26701 Cryptocurrency Market Cap
26702 Cryptocurrency Market Cap
26703 Venezuela-Guyana Conflict
26704 Bloomberg's Billionaires Index
26705 Apple iPhone Development
26706 Stripe IPO
26700 65 The current market capitalization of cryptocurrencies is at $2.35 trillion. Given the volatility of the cryptocurrency market, there is a reasonable chance it could dip below $1.5 trillion, but the threshold is relatively low. Various macroeconomic factors, regulatory changes, and technological developments will likely influence the market. Therefore, a moderate probability of 65% is assigned, allowing for both growth and potential declines in market cap by September 29, 2024.
26701 40 With the current market capitalization already at $2.35 trillion, a reduction to below $2 trillion is plausible but not the most likely scenario given historical volatility patterns and market dynamics. Considering this, a probability of 40% is reasonable as it acknowledges both potenti

## Upload

In [21]:
uploads(ifps)

Prediction posted for  26700
Comment posted for  26700
Prediction posted for  26701
Comment posted for  26701
Prediction posted for  26702
Comment posted for  26702
Prediction posted for  26703
Comment posted for  26703
Prediction posted for  26704
Comment posted for  26704
Prediction posted for  26705
Comment posted for  26705
Prediction posted for  26706
Comment posted for  26706


In [22]:
len(today_ids)

7